# 📊 JupyterHealth Demo: AGP Report

Sample AGP report dashboard

See the `researcher-view-cgm` notebook for a walkthrough of this analysis

---


In [ ]:
## 🔗 Step 1: Connect to JupyterHealth Exchange and Explore

# We use the `jupyterhealth-client` library to securely pull CGM data from a trusted JupyterHealth Data Exchange. 

In [ ]:
import json
import os
from enum import Enum
from pathlib import Path
from urllib.parse import parse_qs

import cgmquantify
import pandas as pd
import requests
from IPython.display import Markdown, display
from jupyterhealth_client import Code, JupyterHealthClient


# Convenience function to display markdown
def md(s):
    display(Markdown(s))


CGM = Code.BLOOD_GLUCOSE

pd.options.mode.chained_assignment = None

jh_client = JupyterHealthClient()

# Use "if not VOILA:" blocks to print debug/private output
# Otherwise all output is by default shown by VOILA
VOILA = os.environ.get("VOILA_REQUEST_URL")
query_string = os.environ.get('QUERY_STRING', '')
url_parameters = parse_qs(query_string)


In [ ]:
# pick patient id, study id
study_id = 30012

def _fmt_fhir_name(res: dict):
    name = res['name'][0]
    given = " ".join(name['given'])
    return f"{given} {name['family']}, id: {res['id']}"

if VOILA and "smart" in url_parameters:
    token_file = Path(os.environ["SMART_TOKEN_FILE"])
    with token_file.open() as f:
        token_info = json.load(f)
        smart_config = token_info["smart_config"]
        fhir_url = token_info["fhir_url"]
        token_response = token_info["token"]
        fhir_token = token_response["access_token"]
    
    fhir = requests.Session()
    fhir.headers = {"Authorization": f"Bearer {fhir_token}"}
    r = fhir.get(fhir_url + token_response["profile"]["reference"])
    if not r.ok:
        print(r.text)
        r.raise_for_status()
    practitioner = r.json()
    r = fhir.get(fhir_url + f"Patient/{token_response['patient']}")
    if not r.ok:
        print(r.text)
        r.raise_for_status()
    fhir_patient = r.json()
    patient = jh_client.get_patient_by_external_id(fhir_patient["id"])
    patient_id = patient["id"]
    md(f"""
**Using SMART launch**

**Practitioner**: {_fmt_fhir_name(practitioner)} 
    """)
else:
    fhir_patient = None
    patient_id = 40037
    
    practitioner = jh_client.get_user()
    patient = jh_client.get_patient(patient_id)
    md(f"""
**Practitioner**: User/{practitioner["id"]} {practitioner["firstName"]} {practitioner["lastName"]} {practitioner["email"]} 
    """)

In [ ]:
df = jh_client.list_observations_df(
    patient_id=patient_id, study_id=study_id, limit=10_000, code=CGM,
)

if not VOILA:
    df.head()

In [ ]:
def patient_info():
    if fhir_patient:
        md(f"""**Patient**: {_fmt_fhir_name(fhir_patient)}; **DOB:** {fhir_patient['birthDate']}""")
    else:
        md(f"""**Patient**: {patient["nameGiven"]} {patient["nameFamily"]}; **DOB:** {patient['birthDate']}""")
    start = df.effective_time_frame_date_time.min()
    end = df.effective_time_frame_date_time.max()
    days = (end - start).total_seconds() // (3600 * 24)
    if start.year == end.year:
        start_s = start.strftime("%B %-d")
    else:
        start_s = start.strftime("%B %-d, %Y")
    if start.year == end.year and start.month == end.month:
        end_s = end.strftime("%-d, %Y")
    else:
        end_s = end.strftime("%B %-d, %Y")

    md(f"**{days:.0f} days:** {start_s} - {end_s}")
    md("---")
    

if not VOILA:
    patient_info()

In [ ]:
# #### The first block of code:
# 	1.	✅ Confirms units are in mg/dL.
# 	2.	🔍 Extracts just CGM glucose values and timestamps.
# 	3.	🕐 Sorts the glucose values by time for downstream analysis.

In [ ]:
# Reduce data to relevant subset for cgm
assert (df.blood_glucose_unit == "MGDL").all()
# reduce data
cgm = df.loc[
    :,
    [
        "blood_glucose_value",
        "effective_time_frame_date_time_local",
    ],
]
# ensure sorted by date
cgm = cgm.sort_values("effective_time_frame_date_time_local")

In [ ]:
# Mean of glucose
mean_glucose = cgm["blood_glucose_value"].mean()

# Standard Deviation (SD) of glucose
std_glucose = cgm["blood_glucose_value"].std()


def display_mean_glucose():
    # Coefficient of Variation (CV) = (SD / Mean) * 100
    cv_glucose = (std_glucose / mean_glucose) * 100
    
    # Print results
    md(f"**Mean Glucose:** {mean_glucose:.2f} mg/dL")
    md(f"**Standard Deviation (SD):** {std_glucose:.2f} mg/dL")
    md(f"**Coefficient of Variation (CV):** {cv_glucose:.2f}%")

In [ ]:
# Compute Glucose Management Indicator (GMI)
def display_gmi():
    gmi = 3.31 + (0.02392 * mean_glucose)
    
    # Print result
    md(f"**Glucose Management Indicator (GMI):** {gmi:.2f}%")

In [ ]:
# CGM and AGP Visualization

# 1. Imports
import datetime

import matplotlib.pyplot as plt
import pandas as pd

# 2. Enums and Utility Mappings


class Category(Enum):
    very_low = "Very Low"
    low = "Low"
    target_range = "Target Range"
    high = "High"
    very_high = "Very High"


category_colors = {
    Category.very_low.value: "#a00",
    Category.low.value: "#f44",
    Category.target_range.value: "#CDECCD",
    Category.high.value: "#FDBE85",
    Category.very_high.value: "#FD8D3C",
}

# 3. Data Preparation
cgm_plot = cgm.copy()
cgm_plot["effective_time_frame_date_time_local"] = pd.to_datetime(
    cgm_plot["effective_time_frame_date_time_local"]
)
cgm_plot["date"] = cgm_plot["effective_time_frame_date_time_local"].dt.date.astype(str)
cgm_plot["hour"] = (
    cgm_plot["effective_time_frame_date_time_local"].dt.hour
    + cgm_plot["effective_time_frame_date_time_local"].dt.minute / 60
)

# 4. Classification Function


def classify_glucose(row):
    if row.blood_glucose_value < 54:
        return Category.very_low.value
    elif row.blood_glucose_value < 70:
        return Category.low.value
    elif row.blood_glucose_value < 180:
        return Category.target_range.value
    elif row.blood_glucose_value < 250:
        return Category.high.value
    else:
        return Category.very_high.value


cgm_plot["category"] = cgm_plot.apply(classify_glucose, axis=1)

# 5. Helper to Add Background Zones


def add_glucose_zones(ax):
    ax.axhspan(0, 54, facecolor="#a00", alpha=0.1)
    ax.axhspan(54, 70, facecolor="#f44", alpha=0.1)
    ax.axhspan(70, 180, facecolor="#CDECCD", alpha=0.3)
    ax.axhspan(180, 250, facecolor="#FDBE85", alpha=0.2)
    ax.axhspan(250, 400, facecolor="#FD8D3C", alpha=0.2)

In [ ]:
# 6. Daily Subplots
def daily_subplots():
    cols = 5
    dates = sorted(cgm_plot["date"].unique())[:10]
    rows = (len(dates) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 3), sharey=True)
    axes = axes.flatten()
    grouped = cgm_plot.groupby("date")
    
    for i, date in enumerate(dates):
        ax = axes[i]
        day_data = grouped.get_group(date).sort_values("hour")
        add_glucose_zones(ax)
    
        smoothed = (
            day_data["blood_glucose_value"]
            .rolling(window=5, center=True, min_periods=1)
            .median()
        )
        ax.plot(day_data["hour"], smoothed, color="green", label="Median Glucose")
    
        # Fill high
        ax.fill_between(
            day_data["hour"],
            180,
            day_data["blood_glucose_value"],
            where=day_data["blood_glucose_value"] > 180,
            color="#FDAE61",
            alpha=0.5,
        )
        # Plot low
        low_mask = day_data["blood_glucose_value"] < 70
        ax.plot(
            day_data["hour"][low_mask],
            day_data["blood_glucose_value"][low_mask],
            color="#E41A1C",
            linewidth=1.5,
        )
    
        weekday = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%A")
        ax.set_xlim(0, 24)
        ax.set_xticks([12])
        ax.set_xticklabels(["12 PM"])
        ax.set_ylim(0, 300)
        ax.set_yticks([54, 70, 180, 250, 300])
        ax.set_yticklabels(["", "70", "180", "250", "300"])
        ax.set_title(f"{weekday}\n{date}")
        ax.axvline(x=12, color="gray", linestyle="--", linewidth=1, alpha=0.6)
        ax.grid(True)
        if i == 0:
            ax.legend(fontsize=8, loc="upper right", frameon=False)
    
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")
    
    fig.suptitle("Daily CGM Profiles with Glucose Zones", fontsize=16, y=0.94)
    fig.set_size_inches(15, 5)
    
    plt.tight_layout()
    plt.subplots_adjust(
        top=0.75,  # space for the title
        hspace=0.75,  # vertical space between rows (lower = tighter)
        wspace=0,  # horizontal space between columns (try 0 for near-touching)
    )
    plt.show()

In [ ]:
# 7. AGP (Ambulatory Glucose Profile)
def plot_agp():
    cgm_plot["minute_of_day"] = (
        cgm_plot["effective_time_frame_date_time_local"].dt.hour * 60
        + cgm_plot["effective_time_frame_date_time_local"].dt.minute
    )
    cgm_plot["time_bin"] = (cgm_plot["minute_of_day"] // 5) * 5
    
    agp_summary = (
        cgm_plot.groupby("time_bin")["blood_glucose_value"]
        .quantile([0.05, 0.25, 0.5, 0.75, 0.95])
        .unstack()
    )
    agp_summary.columns = ["p5", "p25", "p50", "p75", "p95"]
    agp_summary = agp_summary.reset_index()
    agp_summary["hour"] = agp_summary["time_bin"] / 60
    
    plt.figure(figsize=(12, 6))
    add_glucose_zones(plt)
    plt.fill_between(
        agp_summary["hour"],
        agp_summary["p5"],
        agp_summary["p95"],
        color="#D8E3E7",
        alpha=0.5,
        label="5–95% Range",
    )
    plt.fill_between(
        agp_summary["hour"],
        agp_summary["p25"],
        agp_summary["p75"],
        color="#B0C4DE",
        alpha=0.6,
        label="25–75% Range",
    )
    plt.plot(
        agp_summary["hour"], agp_summary["p50"], color="green", linewidth=2, label="Median"
    )
    
    plt.title("Ambulatory Glucose Profile (AGP)")
    plt.xlabel("Time of Day")
    plt.ylabel("Glucose (mg/dL)")
    plt.xticks([0, 6, 12, 18, 24], ["12 AM", "6 AM", "12 PM", "6 PM", "12 AM"])
    plt.xlim(0, 24)
    plt.ylim(0, 300)
    plt.yticks([54, 70, 180, 250, 300])
    plt.grid(True)
    plt.legend(loc="upper right", fontsize=10, frameon=True, framealpha=0.9)
    plt.tight_layout()
    plt.show()

In [ ]:
# Count total readings
def time_in_range():
    total = len(cgm_plot)
    
    # Compute % time in each range
    tir_data = {
        "Very Low (<54)": (cgm_plot["blood_glucose_value"] < 54).sum() / total * 100,
        "Low (54–69)": (
            (cgm_plot["blood_glucose_value"] >= 54) & (cgm_plot["blood_glucose_value"] < 70)
        ).sum()
        / total
        * 100,
        "Target (70–180)": (
            (cgm_plot["blood_glucose_value"] >= 70)
            & (cgm_plot["blood_glucose_value"] <= 180)
        ).sum()
        / total
        * 100,
        "High (181–250)": (
            (cgm_plot["blood_glucose_value"] > 180)
            & (cgm_plot["blood_glucose_value"] <= 250)
        ).sum()
        / total
        * 100,
        "Very High (>250)": (cgm_plot["blood_glucose_value"] > 250).sum() / total * 100,
    }
    
    # Plot
    plt.figure(figsize=(8, 5))
    bars = plt.barh(
        list(tir_data.keys()),
        list(tir_data.values()),
        color=["#a00", "#f44", "#CDECCD", "#FDBE85", "#FD8D3C"],
    )
    
    plt.xlabel("Time in Range (%)")
    plt.title("Time in Glucose Ranges")
    plt.xlim(0, 100)
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    
    # Add percentage labels
    for bar in bars:
        width = bar.get_width()
        plt.text(
            width + 1,
            bar.get_y() + bar.get_height() / 2,
            f"{width:.1f}%",
            va="center",
            fontsize=9,
        )
    
    plt.tight_layout()
    plt.show()

In [ ]:
def time_in_range_vertical():
    # Define the same time-in-range calculation
    tir_data = {
        "Very Low": (cgm_plot["blood_glucose_value"] < 54).sum(),
        "Low": (
            (cgm_plot["blood_glucose_value"] >= 54) & (cgm_plot["blood_glucose_value"] < 70)
        ).sum(),
        "Target": (
            (cgm_plot["blood_glucose_value"] >= 70)
            & (cgm_plot["blood_glucose_value"] <= 180)
        ).sum(),
        "High": (
            (cgm_plot["blood_glucose_value"] > 180)
            & (cgm_plot["blood_glucose_value"] <= 250)
        ).sum(),
        "Very High": (cgm_plot["blood_glucose_value"] > 250).sum(),
    }
    
    total = sum(tir_data.values())
    tir_pct = {k: v / total * 100 for k, v in tir_data.items()}
    
    # Colors that match your other plots
    tir_colors = {
        "Very Low": "#a00",
        "Low": "#f44",
        "Target": "#CDECCD",
        "High": "#FDBE85",
        "Very High": "#FD8D3C",
    }
    
    # Plot the vertical stacked bar
    plt.figure(figsize=(2, 6))
    
    bottom = 0
    for label in ["Very Low", "Low", "Target", "High", "Very High"]:
        height = tir_pct[label]
        plt.bar(
            0, height, bottom=bottom, color=tir_colors[label], width=0.5, edgecolor="white"
        )
        # Add percent labels
        if height > 3:  # skip labeling tiny slivers
            plt.text(0.6, bottom + height / 2, f"{height:.0f}%", va="center", fontsize=9)
        bottom += height
    
    # Formatting
    plt.xlim(-0.5, 2)
    plt.ylim(0, 100)
    plt.xticks([])
    plt.ylabel("Time in Range (%)")
    plt.title("Glucose Zones")
    
    # Optional: add reference lines
    for y in [70, 180, 250]:  # these are just visual references, can be removed
        plt.axhline(y=y, color="gray", linestyle="--", linewidth=0.5, alpha=0.3)
    
    plt.box(False)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_glucose_bounds():
    df["Time"] = df.effective_time_frame_date_time_local
    df["Glucose"] = df.blood_glucose_value
    df["Day"] = df["Time"].dt.date
    cgmquantify.plotglucosebounds(df)

In [ ]:
def mage():
    # Calculate differences between consecutive glucose values
    cgm["glucose_diff"] = cgm["blood_glucose_value"].diff().abs()
    
    # Define a threshold for significant glucose excursions (e.g., 1 SD)
    std_glucose = cgm["blood_glucose_value"].std()
    threshold = std_glucose
    
    # Compute MAGE: Mean of all large excursions above the threshold
    mage = cgm[cgm["glucose_diff"] > threshold]["glucose_diff"].mean()
    md(f"**Mean Amplitude of Glycemic Excursions (MAGE):** {mage:.2f} mg/dL")

In [ ]:
def modd():
    # Extract hour and minute from the timestamp
    cgm["time_of_day"] = cgm["effective_time_frame_date_time_local"].dt.strftime("%H:%M")
    
    # Compute the mean glucose value for each time of day across all days
    mean_per_time = cgm.groupby("time_of_day")["blood_glucose_value"].mean()
    
    # Compute MODD as the mean of absolute day-to-day differences
    modd = mean_per_time.diff().abs().mean()
    md(f"**Mean of Daily Differences (MODD):** {modd:.2f} mg/dL")

In [ ]:
# Layout the actual dashboard

from ipywidgets import HBox, Output, VBox

time_in_range_out = Output()
with time_in_range_out:
    # time_in_range()
    time_in_range_vertical()

info_out = Output()
with info_out:
    patient_info()
    display_mean_glucose()
    display_gmi()
    mage()
    modd()

agp_out = Output()
with agp_out:
    plot_agp()

daily_out = Output()
with daily_out:
    daily_subplots()

display(
    VBox(
        [
            HBox(
                [
                    time_in_range_out,
                    info_out,
                ]
            ),
            agp_out,
            daily_out,
        ]
    )
)